- 수집할 페이지의 주소
    - https://pjt3591oo.github.io/

- user agent 확인
   * http://m.avalon.co.kr/check.html

In [24]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os

# 출력화면 초기화 하는 모듈
from IPython.display import clear_output

### 요청하는 함수

In [25]:
def getSource(site):
    # 헤더 정보
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36',   
    }
    
    # 요청한다.
    response = requests.get(site, headers = header_info)
#     print(response.text)

    # bs4 객체 생성
    soup = bs4.BeautifulSoup(response.text,'lxml')
    
    return soup

In [12]:
site = 'https://pjt3591oo.github.io/'

# getSource(site)

### 한 페이지의 데이터를 수집해 저장하는 함수

In [20]:
def getData(soup):
    # 데이터가 있는 영역 전체를 가져온다.
    a1 = soup.select_one('body > main > div > div')
    
    # 데이터가 있는 p태그를 가져온다
    # a2 = a1.select('div.p') # 위에서 중복되는 부분 제거
    a2 = a1.select('body > main > div > div > div)
    
    # 데이터를 담을 딕셔너리
    data_dict ={
        '제목':[],
        '소제목':[],
        '작성일자':[],
        '작성자':[]
    }
    
    # div 태그의 수만큼 반복한다.
    for div in a2:
        # 큰 제목 데이터
        a3 = div.select_one('h3 > a')
        data1 = a3.text.strip()
        
        # 소제목 데이터
        a4 = div.select_one('h4')
        data2 = a4.text.strip()
        # 만약 빈 문자열이 추출됬다면 결측치로 변경
        if len(data2) == 0:
            data2 = np.nan
        
        
        # 작성일자와 작성자 데이터
        a5 = div.select_one('p > span')
        data3 = a5.text.strip()
        data4 = data3.split('|') 
        data5 = data4[0].strip() # 작성일자
        data6 = data4[1].strip() # 작성자
        
        # 딕셔너리에 데이터를 담는다.
        data_dict['제목'].append(data1)
        data_dict['소제목'].append(data2)
        data_dict['작성일자'].append(data5)
        data_dict['작성자'].append(data6)
        
        
    # 데이터 프레임 생성
    df1 = pd.DataFrame(data_dict)
     
    # 데이터 프레임 저장
    if os.path.exists('data1.csv') == False:
        # 파일이 없을 경우
        df1.to_csv('data1.csv', encoding='utf-8-sig', index=False) # 액셀에서 글자깨짐 방지
    else:
        df1.to_csv('data1.csv', encoding='utf-8-sig', index=False, header=False, mode='a') # 기존꺼 뒤에 추가해서 저장

### 다음페이지 존재 여부 확인하는 함수

In [21]:
def getNextPage(soup):
    # next 버튼 태그를 가져온다.
    a1 = soup.select_one('body > main > div > div > div.pagination > ul > li:nth-child(6) > a')

    
    # 만약 가져온 것이 있다면
    if a1 != None:
        # a 태그의 href 속성 값을 가져온다.
        href = a1.attrs['href']
        return href
    else:
        return None

In [26]:
site = 'https://pjt3591oo.github.io/'

soup = getSource(site)
getNextPage(soup)

'/page2/'

In [22]:
site = 'https://pjt3591oo.github.io'
page = ''

while True:
    # 딜레이
    time.sleep(1)
    
    # 기존 출력된거 정리
    clear_output(wait=True)
    
    # 페이지 요청
    soup = getSource(site+page)
    
    # 데이터 수집
    getData(soup)
    
    # 다음페이지 존재확인
    page = getNextPage(soup)
    
    # 다음페이지가 없다면 중단
    if page == None:
        print('수집완료')
        break

수집완료
